# FHIR Client Experiments

Here fhirclient is being tested to connect to a FHIR-based terminology server by LOINC at
https://fhir.loinc.org. 

The LOINC terminology server is based on smile CDR, a vendor clinical data repository. 
Underneath the cover, as of September 15th, 2019, it's at BETA release, uses HAPI FHIR (4.0.0-snapshot), and 
the endpoints emit FHIR 3.0.1.

All requests to LOINC's FHIR terminology server require basic authentication with a LOINC 
username and password. 

### Install required packages

In [1]:
!pip install fhirclient

## Basic Authentication

fhirclient doesn't support basic authenticatio which is required by the LOINC FHIR server. 

Below is an extension of fhirclient.auth.FHIRAuth to support basic authentication. 

In [2]:
from fhirclient.auth import FHIRAuth
from base64 import b64encode

class FHIRBasicAuth(FHIRAuth):
  auth_type = 'basic'
  
  def __init__(self, state=None, username=None, password=None):
    if username is not None and password is not None:
      self.logon(username, password)
    else:
      self._user_pass = None
    super(FHIRBasicAuth, self).__init__(state=state)    
  
  @property
  def ready(self):
    return True if self._user_pass is not None else False
  
  def reset(self):
    super(FHIRBasicAuth, self).reset()
    self._user_pass = None
    
  def logon(self, username, password):
    self._user_pass = b64encode(bytes(f"{username}:{password}", encoding='utf-8')).decode("ascii")
  
  def can_sign_headers(self):
    return True if self._user_pass is not None else False
  
  def signed_headers(self, headers):
    """ Returns updated HTTP headers. Raises if there is no username/password"""
    if not self.can_sign_headers():
      raise Exception("Cannot sign headers since there is no username/password")
    
    headers = headers or {}
    headers['Authorization'] = f"Basic {self._user_pass}"
    
    return headers 
# register the class. Only need to run once. 
try:
  FHIRBasicAuth.register()
except Exception as e:
  print(f"Failed to register FHIRBasicAuth: {str(e)}")

The FHIRServer by default only instantiate a FHIRAuth object *after* reading the CapabilityStatemnt 
(<fhir_base_uri>/metadata). So we need to add a server implementation that would accept FHIRAuth when 
it is created. 

In [3]:
from fhirclient.server import FHIRServer
class FHIRAuthServer(FHIRServer):
  def __init__(self, client, base_uri=None, state=None, auth=None):
    super(FHIRAuthServer, self).__init__(client, base_uri, state)
    self.auth = auth

Now it's ready to read from the LOINC FHIR server. First get the username and password from user. 

In [4]:
from getpass import getpass 

username = input("Username: ")
password = getpass("Password: ")

## The first issue: Invalid FHIR response

Try to reach the CodeSystem loinc at https://fhir.loinc.org/CodeSystem/loinc. The FHIR api returns the 
content. However, the data returned doesn't conform to the FHIR 3.0.1 standard. fhirclient throws a
FHIRValidationError. 

The problem is that in the response from the LOINC FHIR server, the first two CodeSystem.property has no required "type" field. 

The data looks like below. As you can see, in the first two "property", there's no "type", which makes them invalid.  

```
{
  resourceType: "CodeSystem",
  ...
  property: [
    {
      code: "parent",
      uri: "http://hl7.org/fhir/concept-properties#parent",
      description: "A parent code in the Multiaxial Hierarchy"
    },
    {
      code: "child",
      uri: "http://hl7.org/fhir/concept-properties#child",
      description: "A child code in the Multiaxial Hierarchy"
    },
    {
      code: "STATUS",
      uri: "http://loinc.org/property/STATUS",
      description: "Status of the term. Within LOINC, codes with STATUS=DEPRECATED are considered inactive. Current values: ACTIVE, TRIAL, DISCOURAGED, and DEPRECATED",
      type: "string"
    },
    ...
  ]
}
```

Here's a code snippet that catches the error and fail to instantiate an object from the response. 

In [5]:
from fhirclient.models.fhirabstractbase import FHIRValidationError

auth = FHIRBasicAuth(None, username, password)
fhir = FHIRAuthServer(None, base_uri='https://fhir.loinc.org', state=None, auth=auth)
fhir.prepare() 
if fhir.ready:
  import fhirclient.models.codesystem as c
  try:
    loinc = c.CodeSystem.read('loinc', fhir)
  except FHIRValidationError as e:
    print(f"Validation error: {str(e)}")
  except: 
    print("a different error")
  print(f"Title: {loinc.title}")
  print(f"Description: {loinc.description}")
else:
  print(f"Server is not ready")



Validation error: {root}:
  property.0:
    'Non-optional property "type" on <fhirclient.models.codesystem.CodeSystemProperty object at 0x102f6a450> is missing'


NameError: name 'loinc' is not defined